## PySpark

### Pandas-Friendly Big Data Processing with Spark

In [ ]:
!pip install "pyspark[pandas_on_spark]"

Spark enables scaling of your pandas workloads across multiple nodes. However, learning PySpark syntax can be daunting for pandas users. 

Pandas API on Spark enables leveraging Spark's capabilities for big data while retaining a familiar pandas-like syntax.

The following code compares the syntax between PySpark and the Pandas API on Spark.

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)


Pandas API on Spark:

In [ ]:
import numpy as np
import pyspark.pandas as ps


In [ ]:
psdf = ps.DataFrame(
    {
        "A": ["foo", "bar", "foo"],
        "B": ["one", "one", "two"],
        "C": [0.1, 0.3, 0.5],
        "D": [0.2, 0.4, 0.6],
    }
)


In [ ]:
psdf.sort_values(by='B')


,A,B,C,D
0,foo,one,0.1,0.2
1,bar,one,0.3,0.4
2,foo,two,0.5,0.6


In [ ]:
psdf.groupby('A').sum()


,C,D
A,,
foo,0.6,0.8
bar,0.3,0.4


In [ ]:
psdf.query("C > 0.4")

,A,B,C,D
2,foo,two,0.5,0.6


In [ ]:
psdf[["C", "D"]].abs()

,C,D
0,0.1,0.2
1,0.3,0.4
2,0.5,0.6


PySpark:

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import abs
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark_data = spark.createDataFrame([
    ("foo", "one", 0.1, 0.2),
    ("bar", "one", 0.3, 0.4),
    ("foo", "two", 0.5, 0.6),
], ["A", "B", "C", "D"])

In [ ]:
spark_data.sort(col('B')).show()

+---+---+---+---+
|  A|  B|  C|  D|
+---+---+---+---+
|foo|one|0.1|0.2|
|bar|one|0.3|0.4|
|foo|two|0.5|0.6|
+---+---+---+---+



In [ ]:
spark_data.groupBy('A').sum().show()

+---+------+------+
|  A|sum(C)|sum(D)|
+---+------+------+
|foo|   0.6|   0.8|
|bar|   0.3|   0.4|
+---+------+------+



In [ ]:
spark_data.filter(col('C') > 0.4).show()

+---+---+---+---+
|  A|  B|  C|  D|
+---+---+---+---+
|foo|two|0.5|0.6|
+---+---+---+---+



In [ ]:
spark_data.select(abs(spark_data["C"]).alias("C"), abs(spark_data["D"]).alias("D"))


DataFrame[C: double, D: double]

### PySpark SQL: Enhancing Reusability with Parameterized Queries

In [ ]:
!pip install "pyspark[sql]"

In PySpark, parametrized queries enable the same query structure to be reused with different inputs, without rewriting the SQL.

Additionally, they safeguard against SQL injection attacks by treating input data as parameters rather than as executable code.

In [2]:
from pyspark.sql import SparkSession
import pandas as pd 

spark = SparkSession.builder.getOrCreate()

In [4]:
# Create a Spark DataFrame
item_price_pandas = pd.DataFrame({"item_id": [1, 2, 3, 4], "price": [4, 2, 5, 1]})
item_price = spark.createDataFrame(item_price_pandas)
item_price.show()

+-------+-----+
|item_id|price|
+-------+-----+
|      1|    4|
|      2|    2|
|      3|    5|
|      4|    1|
+-------+-----+



In [16]:
query = """SELECT item_id, price 
FROM {item_price} 
WHERE item_id = {id_val} 
"""

spark.sql(query, id_val=1, item_price=item_price).show()

+-------+-----+
|item_id|price|
+-------+-----+
|      1|    4|
+-------+-----+



In [17]:
spark.sql(query, id_val=2, item_price=item_price).show()

+-------+-----+
|item_id|price|
+-------+-----+
|      2|    2|
+-------+-----+

